Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StatisticalRethinking
using CmdStan, StanMCMCChain
gr(size=(500,500));

CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

### snippet 4.7

In [3]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [4]:
df2 = filter(row -> row[:age] >= 18, df)
mean_weight = mean(df2[:weight])
df2[:weight_c] = convert(Vector{Float64}, df2[:weight]) .- mean_weight ;

Define the Stan language model

In [5]:
weightsmodel = "
data {
 int < lower = 1 > N; // Sample size
 vector[N] height; // Predictor
 vector[N] weight; // Outcome
}

parameters {
 real alpha; // Intercept
 real beta; // Slope (regression coefficients)
 real < lower = 0 > sigma; // Error SD
}

model {
 height ~ normal(alpha + weight * beta , sigma);
}

generated quantities {
}
";

Define the Stanmodel and set the output format to :mcmcchain.

In [6]:
stanmodel = Stanmodel(name="weights", monitors = ["alpha", "beta", "sigma"],model=weightsmodel,
  output_format=:mcmcchain);
# Input data for cmdstan
heightsdata = Dict("N" => length(df2[:height]), "height" => df2[:height], "weight" => df2[:weight_c]);

Sample using cmdstan

In [7]:
rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  summary=false, CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
         Mean        SD        Naive SE       MCSE      ESS
alpha 154.592636 0.27728853 0.00438431654 0.0041170484 1000
 beta   0.904585 0.04196428 0.00066351353 0.0006410256 1000
sigma   5.101437 0.19260099 0.00304528910 0.0032650470 1000

Quantiles:
          2.5%       25.0%       50.0%       75.0%        97.5%   
alpha 154.0559750 154.4040000 154.5930000 154.7800000 155.13202500
 beta   0.8244843   0.8755065   0.9045675   0.9326070   0.98624502
sigma   4.7504543   4.9683075   5.0939800   5.2269975   5.49581850



Save the chains in a JLS file

In [8]:
serialize("m4.4s.jls", chn)

chn2 = deserialize("m4.4s.jls")

Object of type "Chains{Float64}"

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Union{Missing, Float64}[154.434 0.895018 4.68149; 154.748 0.990428 5.05521; … ; 154.538 0.95415 4.78944; 154.78 0.95049 5.00545]

Union{Missing, Float64}[154.4 0.878419 5.22697; 154.636 0.895923 4.87384; … ; 154.822 0.969291 4.86428; 154.397 0.850408 5.12031]

Union{Missing, Float64}[154.145 0.962201 4.96052; 155.101 0.846375 5.05321; … ; 154.319 0.885561 4.95493; 154.735 0.877993 5.0922]

Union{Missing, Float64}[154.6 0.944875 5.05308; 154.385 0.93459 5.29889; … ; 154.629 0.939917 5.3444; 154.598 0.860162 4.92543]

Should be identical to earlier result

In [9]:
describe(chn2)

Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
         Mean        SD        Naive SE       MCSE      ESS
alpha 154.592636 0.27728853 0.00438431654 0.0041170484 1000
 beta   0.904585 0.04196428 0.00066351353 0.0006410256 1000
sigma   5.101437 0.19260099 0.00304528910 0.0032650470 1000

Quantiles:
          2.5%       25.0%       50.0%       75.0%        97.5%   
alpha 154.0559750 154.4040000 154.5930000 154.7800000 155.13202500
 beta   0.8244843   0.8755065   0.9045675   0.9326070   0.98624502
sigma   4.7504543   4.9683075   5.0939800   5.2269975   5.49581850



End of `m4.4s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*